<a href="https://colab.research.google.com/github/rockingkamal/NLP/blob/main/Creation_of_Objective_Questions_with__Multiple_Correct_Answers_using_generative_text_models_NLP_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Packages to install

In [2]:
!pip install -U langchain
!pip install PyPDF2
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## Libraries

In [3]:

import PyPDF2
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os


## import OPENAI API key

In [16]:
os.environ['OPENAI_API_KEY']="sk-#*******************#" #paste api here

In [17]:
from PyPDF2 import PdfReader

reader = PdfReader("/content/chapter-3.pdf") #choose data set path
chapter=''
number_of_pages = len(reader.pages)
for i in range(number_of_pages):
  page = reader.pages[i]
  text = page.extract_text()
  text=text.replace('\n'," ")
  chapter=chapter+f' {text}'
chapter=" ".join(chapter.split())

In [23]:
#total number of pages
print(len(reader.pages))

13


In [39]:
#print(chapter)

Fig. 1 – Robert Clive accepting the Diwani of Bengal, Bihar and Orissa from the Mughal ruler in 1765 The Company Becomes the Diwan On 12 August 1765, the Mughal emperor appointed the East India Company as the Diwan of Bengal. The actual event most probably took place in Robert Clive’s tent, with a few Englishmen and Indians as witnesses. But in the painting above, the event is shown as a majestic occasion, taking place in a grand setting. The painter was commissioned by Clive to record the memorable events in Clive’s life. The grant of Diwani clearly was one such event in British imagination. As Diwan, the Company became the chief financial administrator of the territory under its control. Now it had to think of administering the land and organising its revenue resources. This had to be done in a way that could yield enough revenue to meet the growing expenses of the company. A trading company had also to ensure that it could buy the products it needed and sell what it wanted. Ruling t

In [19]:
print(len(chapter))

26198


In [20]:
print(len(text))

823


In [10]:
#chapter_1=chapter[7:4050]

#prompt to get only one correct option


In [18]:
#prompt to get only one correct option

llm=ChatOpenAI(model='gpt-3.5-turbo',temperature=0.2) #used lastest basic version of chatGPT "gpt-3.5.turbo"

mcq_questions_prompt = PromptTemplate(
    input_variables=['Text'],
    template=
"""
Generate 5 Multiple choice Questions with Medium to Hard level difficulty from the given context with each question having 4 options.
The Format of the Questions and Answers should be

Q1. Question --Each Question should be in a single line.

a. option 1
b. option 2
c. option 3
d. option 4
Correct Options: (a) or (c) or (d) or (b)

Correct Options should contain only one correct options in single with bold letters.
{Text}
"""
)

mcq_Ques_chains=LLMChain(llm=llm, prompt=mcq_questions_prompt, output_key="mcq_questions")

mcq_question_list_prompt = PromptTemplate(
    input_variables=["mcq_questions"],
    template="""
  Please extract only the questions from the following list of generated MCQs and should return as list of [str1,str2,str3...] and name the list as "mca_questions_list"
  {mcq_questions}
  """
)
mcq_Ques_chains_list=LLMChain(llm=llm,prompt=mcq_question_list_prompt,output_key= "mcq_questions_list")

overall_chain = SequentialChain(
    chains=[mcq_Ques_chains, mcq_Ques_chains_list],
    input_variables=["Text"],
    # Here we can return multiple variables
    output_variables=["mcq_questions", "mcq_questions_list"],
    verbose=True)


result=overall_chain({'Text':text})  ## make sure len of text should not be more than 4097
print(result['mcq_questions'])
print('\n')
print(result['mcq_questions_list'])



> Entering new SequentialChain chain...

> Finished chain.
Q1. What were the two problems that arose with the new Munro system of fixing revenue?

a. Increased taxation and lack of transparency
b. Decreased taxation and increased transparency
c. Lack of taxation and lack of transparency
d. Increased taxation and increased transparency

Correct Option: (a)

Q2. Why were ryots reluctant to grow indigo?

a. Indigo was a difficult crop to cultivate
b. Indigo had a low market demand
c. Indigo required a large amount of water for irrigation
d. Indigo cultivation was controlled by the British

Correct Option: (d)

Q3. What were the circumstances that led to the eventual collapse of indigo production in Bengal?

a. The introduction of synthetic dyes
b. The decline in demand for indigo in the international market
c. The uprising of indigo farmers against the British
d. The British government's decision to ban indigo cultivation

Correct Option: (b)

Q4. What was Mahatma Gandhi's role in the C


#promt to print only two right answers



In [12]:
#promt to print only two right answers

llm=ChatOpenAI(model='gpt-3.5-turbo',temperature=0.2)

mcq_questions_prompt = PromptTemplate(
    input_variables=['Text'],
    template=
"""
Generate 5 Multiple choice Questions with Medium to Hard level difficulty from the given context with each question having 4 options out of which atleast 2 options are correct.
Make sure Each question Must Compulsorily have two correct answers.

The Format of the Questions and Answers should be

Q1. Question --Each Question should be in a single line.

a. option 1
b. option 2
c. option 3
d. option 4
Correct Options: (a) & (c) & (d) & (b)

Correct Options should contain only two correct options in single line seperated by & symbol.
{Text}
"""
)

mcq_Ques_chains=LLMChain(llm=llm, prompt=mcq_questions_prompt, output_key="mcq_questions")

mcq_question_list_prompt = PromptTemplate(
    input_variables=["mcq_questions"],
    template="""
  Please extract only the questions from the following list of generated MCQs and should return as list of [str1,str2,str3...] and name the list as "mca_questions_list"
  {mcq_questions}
  """
)
mcq_Ques_chains_list=LLMChain(llm=llm,prompt=mcq_question_list_prompt,output_key= "mcq_questions_list")

overall_chain = SequentialChain(
    chains=[mcq_Ques_chains, mcq_Ques_chains_list],
    input_variables=["Text"],
    # Here we return multiple variables
    output_variables=["mcq_questions", "mcq_questions_list"],
    verbose=True)


result=overall_chain({'Text':text})  ## make sure len of text should not be more than 4097
print(result['mcq_questions'])
print('\n')
print(result['mcq_questions_list'])



> Entering new SequentialChain chain...

> Finished chain.
Q1. What problems did the ryots point out regarding the new Munro system of fixing revenue?

a. Increased taxation
b. Lack of transparency in revenue collection
c. Exploitation by the planter
d. Unfair distribution of land

Correct Options: (a) & (b)

Q2. Why were ryots reluctant to grow indigo?

a. Low profitability of indigo cultivation
b. High demand for other cash crops
c. Lack of knowledge and resources for indigo cultivation
d. Fear of exploitation by the planters

Correct Options: (a) & (c)

Q3. What were the circumstances that led to the eventual collapse of indigo production in Bengal?

a. Decline in demand for indigo in the international market
b. Peasant resistance and protests against indigo cultivation
c. Introduction of more profitable cash crops
d. Government intervention and regulation of indigo industry

Correct Options: (a) & (b)

Q4. What were two problems that arose with the new Munro system of fixing reve

#promt to get more than two correct options


In [9]:
#promt to get more than two correct options

llm=ChatOpenAI(model='gpt-3.5-turbo',temperature=0.2)

mcq_questions_prompt = PromptTemplate(
    input_variables=['Text'],
    template=
"""
Generate 5 Multiple choice Questions with Medium to Hard level difficulty from the given context with each question having 4 options out of which atleast 2 options are correct.
Make sure Each question Must Compulsorily have two correct answers.

The Format of the Questions and Answers should be

Q1. Question --Each Question should be in a single line.

a. option 1
b. option 2
c. option 3
d. option 4
Correct Options: (a) & (c) & (d) & (b)

Correct Options should contain only two correct options in single line seperated by & symbol.
{Text}
"""
)

mcq_Ques_chains=LLMChain(llm=llm, prompt=mcq_questions_prompt, output_key="mcq_questions")

mcq_question_list_prompt = PromptTemplate(
    input_variables=["mcq_questions"],
    template="""
  Please extract only the questions from the following list of generated MCQs and should return as list of [str1,str2,str3...] and name the list as "mca_questions_list"
  {mcq_questions}
  """
)
mcq_Ques_chains_list=LLMChain(llm=llm,prompt=mcq_question_list_prompt,output_key= "mcq_questions_list")

overall_chain = SequentialChain(
    chains=[mcq_Ques_chains, mcq_Ques_chains_list],
    input_variables=["Text"],
    # Here we return multiple variables
    output_variables=["mcq_questions", "mcq_questions_list"],
    verbose=True)


result=overall_chain({'Text':text})  ## make sure len of text should not be more than 4097
print(result['mcq_questions'])
print('\n')
print(result['mcq_questions_list'])



> Entering new SequentialChain chain...

> Finished chain.
Q1. What were the reasons given by the planter to persuade the peasant to grow indigo?

a. Promised higher profits
b. Assured protection from British authorities
c. Offered advanced farming techniques
d. Provided financial assistance for seeds and equipment

Correct Options: (a) & (c) & (d)

Q2. What problems did the peasant point out in growing indigo?

a. Low market demand for indigo
b. Excessive workload and long hours
c. Lack of proper compensation for their labor
d. Inadequate infrastructure for indigo production

Correct Options: (b) & (c) & (d)

Q3. What were two problems that arose with the new Munro system of fixing revenue?

a. Increased tax burden on peasants
b. Unfair distribution of land among peasants
c. Lack of transparency in revenue collection
d. Exploitation of peasants by revenue officials

Correct Options: (a) & (d)

Q4. Why were ryots reluctant to grow indigo?

a. Indigo cultivation required specialized s